***GENERATED CODE FOR crimesregresssion PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DATE Occurance', 'transformation_label': 'String Indexer'}], 'feature': 'DATE Occurance', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': '1/1/2020 0:00', 'max': '9/9/2020 0:00', 'missing': '0', 'distinct': '230'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DATE Occurance'}, {'feature_label': 'DATE Occurance', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DATE Occurance')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'AREA NAME', 'transformation_label': 'String Indexer'}], 'feature': 'AREA NAME', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '77th Street', 'max': 'Wilshire', 'missing': '0', 'distinct': '21'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'AREA NAME'}, {'feature_label': 'AREA NAME', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('AREA NAME')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Crime Description', 'transformation_label': 'String Indexer'}], 'feature': 'Crime Description', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'ARSON', 'max': 'VIOLATION OF RESTRAINING ORDER', 'missing': '0', 'distinct': '51'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Crime Description'}, {'feature_label': 'Crime Description', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Crime Description')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Vict Gender', 'transformation_label': 'String Indexer'}], 'feature': 'Vict Gender', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '420', 'mean': '', 'stddev': '', 'min': 'F', 'max': 'X', 'missing': '80', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Vict Gender'}, {'feature_label': 'Vict Gender', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Vict Gender')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Victim Descent', 'transformation_label': 'String Indexer'}], 'feature': 'Victim Descent', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '420', 'mean': '', 'stddev': '', 'min': 'A', 'max': 'Z', 'missing': '80', 'distinct': '13'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Victim Descent'}, {'feature_label': 'Victim Descent', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Victim Descent')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Premise Description', 'transformation_label': 'String Indexer'}], 'feature': 'Premise Description', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'ABANDONED BUILDING ABANDONED HOUSE', 'max': 'WEBSITE', 'missing': '0', 'distinct': '45'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Premise Description'}, {'feature_label': 'Premise Description', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Premise Description')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Status', 'transformation_label': 'String Indexer'}], 'feature': 'Status', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'AA', 'max': 'IC', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Status'}, {'feature_label': 'Status', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Status')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Status Desc', 'transformation_label': 'String Indexer'}], 'feature': 'Status Desc', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Adult Arrest', 'max': 'Juv Other', 'missing': '0', 'distinct': '5'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Status Desc'}, {'feature_label': 'Status Desc', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Status Desc')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'LOCATION', 'transformation_label': 'String Indexer'}], 'feature': 'LOCATION', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '100 E  3RD                          ST', 'max': 'YUCCA                        ST', 'missing': '0', 'distinct': '479'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'LOCATION'}, {'feature_label': 'LOCATION', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('LOCATION')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.model_selection import train_test_split
from tpot import TPOTRegressor
import pyspark


def functionRegression(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTRegressor(verbosity=3, generations=10, max_time_mins=5,
                              n_jobs=-1, random_state=25, population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Error rate of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run crimesregresssionHooks.ipynb
try:
	#sourcePreExecutionHook()

	crimedatalosangeles = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/Crime_Data_LoS Angeles.csv', 'filename': 'Crime_Data_LoS Angeles.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'domain': 'http://172.31.59.158', 'port': '40070', 'dirPath': '/FileStore/platform', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/'}")
	#sourcePostExecutionHook(crimedatalosangeles)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run crimesregresssionHooks.ipynb
try:
	#transformationPreExecutionHook()

	crimesregresssionautofe = TransformationMain.run(crimedatalosangeles,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "DR_NO", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "201421953.57", "stddev": "1808705.27", "min": "200100628", "max": "211811628", "missing": "0"}, "updatedLabel": "DR_NO"}, {"transformationsData": [{"feature_label": "DATE Occurance", "transformation_label": "String Indexer"}], "feature": "DATE Occurance", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "1/1/2020 0:00", "max": "9/9/2020 0:00", "missing": "0", "distinct": "230"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DATE Occurance"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "TIME Occurance", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1348.3", "stddev": "664.12", "min": "1", "max": "2355", "missing": "0"}, "updatedLabel": "TIME Occurance"}, {"transformationsData": [{"feature_label": "AREA NAME", "transformation_label": "String Indexer"}], "feature": "AREA NAME", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "77th Street", "max": "Wilshire", "missing": "0", "distinct": "21"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "AREA NAME"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Rpt Dist No", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1156.12", "stddev": "581.85", "min": "111", "max": "2198", "missing": "0"}, "updatedLabel": "Rpt Dist No"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Part 1-2", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1.39", "stddev": "0.49", "min": "1", "max": "2", "missing": "0"}, "updatedLabel": "Part 1-2"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Crime  Code", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "511.64", "stddev": "207.85", "min": "110", "max": "956", "missing": "0"}, "updatedLabel": "Crime  Code"}, {"transformationsData": [{"feature_label": "Crime Description", "transformation_label": "String Indexer"}], "feature": "Crime Description", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "ARSON", "max": "VIOLATION OF RESTRAINING ORDER", "missing": "0", "distinct": "51"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Crime Description"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Victim Age", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "28.05", "stddev": "21.89", "min": "0", "max": "91", "missing": "0"}, "updatedLabel": "Victim Age"}, {"transformationsData": [{"feature_label": "Vict Gender", "transformation_label": "String Indexer"}], "feature": "Vict Gender", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "420", "mean": "", "stddev": "", "min": "F", "max": "X", "missing": "80", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Vict Gender"}, {"transformationsData": [{"feature_label": "Victim Descent", "transformation_label": "String Indexer"}], "feature": "Victim Descent", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "420", "mean": "", "stddev": "", "min": "A", "max": "Z", "missing": "80", "distinct": "13"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Victim Descent"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Premise Code", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "443.91", "stddev": "181.64", "min": "101", "max": "801", "missing": "0"}, "updatedLabel": "Premise Code"}, {"transformationsData": [{"feature_label": "Premise Description", "transformation_label": "String Indexer"}], "feature": "Premise Description", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "ABANDONED BUILDING ABANDONED HOUSE", "max": "WEBSITE", "missing": "0", "distinct": "45"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Premise Description"}, {"transformationsData": [{"feature_label": "Status", "transformation_label": "String Indexer"}], "feature": "Status", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "AA", "max": "IC", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Status"}, {"transformationsData": [{"feature_label": "Status Desc", "transformation_label": "String Indexer"}], "feature": "Status Desc", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Adult Arrest", "max": "Juv Other", "missing": "0", "distinct": "5"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Status Desc"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Crm Cd 1", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "484.58", "stddev": "215.72", "min": "121", "max": "956", "missing": "0"}, "updatedLabel": "Crm Cd 1"}, {"transformationsData": [{"feature_label": "LOCATION", "transformation_label": "String Indexer"}], "feature": "LOCATION", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "100 E  3RD                          ST", "max": "YUCCA                        ST", "missing": "0", "distinct": "479"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "LOCATION"}]}))

	#transformationPostExecutionHook(crimesregresssionautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run crimesregresssionHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionRegression(crimesregresssionautofe, ["DR_NO", "TIME Occurance", "Rpt Dist No", "Part 1-2", "Crime  Code", "Victim Age", "Premise Code", "Crm Cd 1", "DATE Occurance_stringindexer", "Crime Description_stringindexer", "Vict Gender_stringindexer", "Victim Descent_stringindexer", "Premise Description_stringindexer", "Status_stringindexer", "Status Desc_stringindexer", "LOCATION_stringindexer"], "AREA NAME_stringindexer")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML ['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    df = df[columnNames]
    R2 = np.round(sklearn.metrics.r2_score(y_test, y_predicted), 1)
    Mean_Squared_Error = np.round(sklearn.metrics.mean_squared_error(y_test, y_predicted), 1)
    Mean_Absolute_Error = np.round(sklearn.metrics.mean_absolute_error(y_test, y_predicted), 1)
    display(" R2 score of Prediction on test data    : %s"%R2)
    display(" Mean Squared Error of Prediction on test data    : %s"%Mean_Squared_Error)
    display(" Mean Absolute Error of Prediction on test data   : %s"%Mean_Absolute_Error)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

